In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from PIL import Image
from torchvision import transforms
import tifffile
import numpy as np

In [ ]:
! pip install Pillow
! pip install --upgrade tifffile imagecodecs
! pip install --force-reinstall tifffile imagecodecs

In [ ]:
! python -m pip install -U imagecodecs[all]

In [ ]:
! pip install rasterio

In [ ]:
import numpy as np
import torch
import torchvision.transforms as transforms
import rasterio

def load_image(file_path, bands=[4, 3, 2]):
    # Open TIF file using rasterio
    with rasterio.open(file_path) as dataset:
        # Read the specified bands of the image
        image_bands = dataset.read(bands).transpose(1, 2, 0)

    # Convert image to float
    # Assuming 16-bit TIF, adjust if needed
    image_bands = image_bands.astype(np.float32) / 65535.0

    # Normalize the image bands with custom mean and std values
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.2, 0.2, 0.2]),
    ])

    input_tensor = transform(image_bands).unsqueeze(0)
    return input_tensor


# Example usage:
file_path = "D:\College files\TISS\Sentinal.tif"
input_image = load_image(file_path)
print("Input image shape:", input_image.shape)

In [10]:
import numpy as np
import torch
import torchvision.transforms as transforms
import rasterio

In [12]:
def load_image(file_path, bands=None):
    # Open TIF file using rasterio
    with rasterio.open(file_path) as dataset:
        # Get the number of bands in the image
        num_bands = dataset.count
        print("Number of bands in the image:", num_bands)

        # If bands are not specified, use the first three bands
        if bands is None:
            bands = list(range(1, min(num_bands, 4) + 1))

        # Read the specified bands of the image
        try:
            image_bands = dataset.read(bands).transpose(1, 2, 0)
        except IndexError:
            raise ValueError("Specified band index is out of range")

    # Convert image to float
    # Assuming 16-bit TIF, adjust if needed
    image_bands = image_bands.astype(np.float32) / 65535.0

    # Normalize the image bands with custom mean and std values
    mean = np.array([0.5] * num_bands)  # Adjust if necessary
    std = np.array([0.2] * num_bands)   # Adjust if necessary
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=mean, std=std),
    ])

    input_tensor = transform(image_bands)
    return input_tensor.unsqueeze(0)




# Example usage:
file_path = "D:\College files\TISS\Sentinal.tif"
input_image = load_image(file_path)
print("Input image shape:", input_image.shape)

Number of bands in the image: 1
Input image shape: torch.Size([1, 1, 846, 1262])


<>:37: SyntaxWarning: invalid escape sequence '\C'
<>:37: SyntaxWarning: invalid escape sequence '\C'
C:\Users\HP\AppData\Local\Temp\ipykernel_11604\3496561127.py:37: SyntaxWarning: invalid escape sequence '\C'
  file_path = "D:\College files\TISS\Sentinal.tif"


In [ ]:
pip install tifffile

In [ ]:
! pip install -U imagecodecs[all]

In [15]:
  import torch
  import torch.nn as nn
  import torch.nn.functional as F
  import tifffile

  class DoubleConv(nn.Module):
      def __init__(self, in_channels, out_channels):
          super(DoubleConv, self).__init__()
          self.double_conv = nn.Sequential(
              nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
              nn.ReLU(inplace=True),
              nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
              nn.ReLU(inplace=True)
          )

      def forward(self, x):
          return self.double_conv(x)
  class UNet(nn.Module):
      def __init__(self, in_channels, out_channels):
          super(UNet, self).__init__()

          self.max_pool_2x2 = nn.MaxPool2d(kernel_size=2, stride=2)
          self.down_conv_1 = DoubleConv(in_channels, 64)  # Update input channels
          self.down_conv_2 = DoubleConv(64, 128)
          self.down_conv_3 = DoubleConv(128, 256)
          self.down_conv_4 = DoubleConv(256, 512)
          self.down_conv_5 = DoubleConv(512, 1024)

          self.up_trans_1 = nn.ConvTranspose2d(
              1024, 512, kernel_size=2, stride=2)
          self.up_conv_1 = DoubleConv(1024, 512)

          self.up_trans_2 = nn.ConvTranspose2d(512, 256, kernel_size=2, stride=2)
          self.up_conv_2 = DoubleConv(512, 256)

          self.up_trans_3 = nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2)
          self.up_conv_3 = DoubleConv(256, 128)

          self.up_trans_4 = nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2)
          self.up_conv_4 = DoubleConv(128, 64)

          self.out = nn.Conv2d(64, out_channels, kernel_size=1)
      def forward(self, x):
          # Encoder
          x1 = self.down_conv_1(x)
          x2 = self.max_pool_2x2(x1)
          x3 = self.down_conv_2(x2)
          x4 = self.max_pool_2x2(x3)
          x5 = self.down_conv_3(x4)
          x6 = self.max_pool_2x2(x5)
          x7 = self.down_conv_4(x6)
          x8 = self.max_pool_2x2(x7)
          x9 = self.down_conv_5(x8)

          # Decoder
          x = self.up_trans_1(x9)
          x = F.interpolate(x, size=x7.size()[2:], mode='bilinear', align_corners=True)
          x = self.up_conv_1(torch.cat([x, x7], 1))

          x = self.up_trans_2(x)
          x = F.interpolate(x, size=x5.size()[2:], mode='bilinear', align_corners=True)
          x = self.up_conv_2(torch.cat([x, x5], 1))

          x = self.up_trans_3(x)
          x = F.interpolate(x, size=x3.size()[2:], mode='bilinear', align_corners=True)
          x = self.up_conv_3(torch.cat([x, x3], 1))

          x = self.up_trans_4(x)
          x = F.interpolate(x, size=x1.size()[2:], mode='bilinear', align_corners=True)
          x = self.up_conv_4(torch.cat([x, x1], 1))

          x = self.out(x)
          return x

  def load_image(file_path):

      image = tifffile.imread(file_path)

      image = image[np.newaxis, ...]


      image_tensor = torch.from_numpy(image).float()

      return image_tensor

  file_path = "D:\College files\TISS\Sentinal.tif"
  image = load_image(file_path)
  model = UNet(in_channels=image.size(1), out_channels=2)
  output = model(image)
  print(output.size())


<>:86: SyntaxWarning: invalid escape sequence '\C'
<>:86: SyntaxWarning: invalid escape sequence '\C'
C:\Users\HP\AppData\Local\Temp\ipykernel_11604\2428076144.py:86: SyntaxWarning: invalid escape sequence '\C'
  file_path = "D:\College files\TISS\Sentinal.tif"
C:\Users\HP\AppData\Local\Temp\ipykernel_11604\2428076144.py:86: SyntaxWarning: invalid escape sequence '\C'
  file_path = "D:\College files\TISS\Sentinal.tif"


RuntimeError: Given groups=1, weight of size [64, 846, 3, 3], expected input[1, 1, 846, 1262] to have 846 channels, but got 1 channels instead

In [10]:

import numpy as np
from sklearn.model_selection import train_test_split

# Load the Sentinel.tif dataset
file_path = "D:\College files\TISS\Sentinal.tif"
dataset = tifffile.imread(file_path)

# Split the dataset into features (X) and labels (y) if applicable
# Assuming you have labels or target values for supervised learning
X = dataset[:, :, :-1]  # Features
y = dataset[:, :, -1]   # Labels

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Alternatively, if you don't have labels, you can split only the features
X_train, X_test = train_test_split(dataset, test_size=0.3, random_state=42)

# Optionally, you can save the split datasets to separate files
# For example, using tifffile.imwrite for TIFF images
tifffile.imwrite("train_set.tif", X_train)
tifffile.imwrite("test_set.tif", X_test)

# Or you can save them as NumPy arrays
np.save("X_train.npy", X_train)
np.save("X_test.npy", X_test)
np.save("y_train.npy", y_train)
np.save("y_test.npy", y_test)

In [15]:
# Load the split datasets with absolute file paths
X_train = tifffile.imread("/content/drive/MyDrive/Sentinal.tif")
X_test = tifffile.imread("/content/drive/MyDrive/Sentinal.tif")
y_train = tifffile.imread("/content/drive/MyDrive/Sentinal.tif")


# Data Preprocessing
def preprocess_data(X, y):
    # Normalization
    X = X / 255.0  # Assuming pixel values range from 0 to 255

    # Convert to torch tensors
    X_tensor = torch.from_numpy(X).float()
    y_tensor = torch.from_numpy(y).long()  # Assuming integer labels

    return X_tensor, y_tensor
# Load the split datasets
X_train = tifffile.imread("/content/drive/MyDrive/Sentinal.tif")
X_test = tifffile.imread("/content/drive/MyDrive/Sentinal.tif")
y_train = tifffile.imread("/content/drive/MyDrive/Sentinal.tif")
y_test = tifffile.imread("/content/drive/MyDrive/Sentinal.tif")

# Preprocess the datasets
X_train, y_train = preprocess_data(X_train, y_train)
X_test, y_test = preprocess_data(X_test, y_test)

# Print the shapes of the preprocessed datasets
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)


X_train shape: torch.Size([846, 1262, 23])
y_train shape: torch.Size([846, 1262, 23])
X_test shape: torch.Size([846, 1262, 23])
y_test shape: torch.Size([846, 1262, 23])


In [17]:

import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm

# Assuming you already have X_train, y_train, X_test, y_test
# Convert the data to PyTorch tensors and create DataLoader objects
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Initialize the model, loss function, and optimizer
model = UNet(in_channels=X_train.size(1), out_channels=2)
# Define the loss function
criterion = nn.BCEWithLogitsLoss()

# Define the optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)



# Define the training function
def train(model, train_loader, criterion, optimizer, num_epochs=10):
    model.train()  # Set the model to train mode
    for epoch in range(num_epochs):
        running_loss = 0.0
        for inputs, targets in tqdm(train_loader, desc=f'Epoch {epoch + 1}/{num_epochs}', leave=False):
            optimizer.zero_grad()  # Zero the parameter gradients
            outputs = model(inputs)  # Forward pass
            loss = criterion(outputs, targets.long())  # Calculate the loss
            loss.backward()  # Backward pass
            optimizer.step()  # Optimize

            running_loss += loss.item() * inputs.size(0)

        epoch_loss = running_loss / len(train_loader.dataset)
        print(f'Train Loss: {epoch_loss:.4f}')

# Train the model
train(model, train_loader, criterion, optimizer, num_epochs=10)


RuntimeError: Given groups=1, weight of size [64, 1262, 3, 3], expected input[1, 32, 1262, 23] to have 1262 channels, but got 32 channels instead

In [25]:

def load_labels(file_path):
    return labels


file_path = "D:\College files\TISS\Sentinal.tif"
image = load_image(file_path)
labels = load_labels(file_path)  # Load binary labels for each pixel

from sklearn.model_selection import train_test_split

# Assuming 'image' is your input image data
X = image
# Assuming 'labels' are your corresponding labels, adjust this according to your data
y = labels

# Define the size of the training and testing sets
test_size = 0.2  # 20% of the data for testing

# Calculate the index to split the data
split_index = int(X.shape[0] * (1 - test_size))
# Split the data into training and testing sets
X_train, X_test = X[:split_index], X[split_index:]
y_train, y_test = y[:split_index], y[split_index:]

# Print the shapes of the split data
print("Training image shape:", X_train.shape)
print("Testing image shape:", X_test.shape)
print("Training labels shape:", y_train.shape)
print("Testing labels shape:", y_test.shape)

TypeError: 'NoneType' object is not subscriptable

In [26]:
def load_labels(file_path):
    return labels


file_path = "D:\College files\TISS\Sentinal.tif"
image = load_image(file_path)
labels = load_labels(file_path)  # Load binary labels for each pixel

from sklearn.model_selection import train_test_split

# Assuming 'image' is your input image data
X = image
# Assuming 'labels' are your corresponding labels, adjust this according to your data
y = labels

# Define the size of the training and testing sets
test_size = 0.2  # 20% of the data for testing

# Calculate the index to split the data
split_index = int(X.shape[0] * (1 - test_size))
# Split the data into training and testing sets
X_train, X_test = X[:split_index], X[split_index:]
y_train, y_test = y[:split_index], y[split_index:]

# Print the shapes of the split data
print("Training image shape:", X_train.shape)
print("Testing image shape:", X_test.shape)
print("Training labels shape:", y_train.shape)
print("Testing labels shape:", y_test.shape)

TypeError: 'NoneType' object is not subscriptable

In [ ]:
# Initialize the model
model = UNet(in_channels=image.size(1), out_channels=2)  # Adjust out_channels if needed

# Define the loss function and optimizer
criterion = nn.BCEWithLogitsLoss()  # Binary cross-entropy loss
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [12]:
import numpy as np
from sklearn.model_selection import train_test_split

def split_data(X, y, test_size=0.2, random_state=42):
    # Flatten the images and labels
    X_flat = X.reshape(X.shape[0], -1)
    y_flat = y.flatten()

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X_flat, y_flat, test_size=test_size, random_state=random_state)

    # Reshape the data back to the original shape
    X_train = X_train.reshape(-1, X.shape[1], X.shape[2], X.shape[3])
    X_test = X_test.reshape(-1, X.shape[1], X.shape[2], X.shape[3])

    return X_train, X_test, y_train, y_test


In [14]:
# Load image and labels
file_path = "D:\College files\TISS\Sentinal.tif"
image  = load_image(file_path)
labels = load_labels(file_path)  # Load binary labels for each pixel

X_train, X_test, y_train, y_test = split_data(image, labels)

# Convert data to PyTorch tensors and create DataLoader objects
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=22, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=22, shuffle=False)

# Initialize the model
model = UNet(in_channels=image.size(1), out_channels=2)  # Adjust out_channels if needed

# Define the loss function and optimizer
criterion = nn.BCEWithLogitsLoss()  # Binary cross-entropy loss
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
def train(model, train_loader, criterion, optimizer, num_epochs=10):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for inputs, targets in tqdm(train_loader, desc=f'Epoch {epoch + 1}/{num_epochs}', leave=False):
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)

        epoch_loss = running_loss / len(train_loader.dataset)
        print(f'Train Loss: {epoch_loss:.4f}')

train(model, train_loader, criterion, optimizer, num_epochs=10)

# Evaluate the model
def evaluate(model, test_loader, criterion):
    model.eval()
    running_loss = 0.0
    with torch.no_grad():
        for inputs, targets in test_loader:
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            running_loss += loss.item() * inputs.size(0)

    test_loss = running_loss / len(test_loader.dataset)
    print(f'Test Loss: {test_loss:.4f}')

# Test the model
evaluate(model, test_loader, criterion)


AttributeError: 'NoneType' object has no attribute 'flatten'

In [ ]:
# Define the rest of your code (loading data, training loop) as before
def load_image(file_path):
    image = tifffile.imread(file_path)

    image = image[np.newaxis, ...]
    image_tensor = torch.from_numpy(image).float()

    return image_tensor

file_path = "/content/drive/MyDrive/Sentinal.tif"
image = load_image(file_path)
model = UNet(in_channels=image.size(1), out_channels=2)
output = model(image)
print(output.size())

In [ ]:
from sklearn.model_selection import train_test_split
dataset = tifffile.imread(file_path)
X = dataset[:, :, :-1]  # Features
y = dataset[:, :, -1]   # Labels
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
import numpy as np
import tifffile

# Shuffle the training dataset
indices = np.arange(X_train.shape[0])
np.random.shuffle(indices)
X_train = X_train[indices]
y_train = y_train[indices]

# Shuffle the testing dataset
indices = np.arange(X_test.shape[0])
np.random.shuffle(indices)
X_test = X_test[indices]
y_test = y_test[indices]

# Saving split dataset
tifffile.imwrite("X_train.tif", X_train)
tifffile.imwrite("X_test.tif", X_test)
tifffile.imwrite("y_train.tif", y_train)
tifffile.imwrite("y_test.tif", y_test)

np.save("X_train.npy", X_train)
np.save("X_test.npy", X_test)
np.save("y_train.npy", y_train)
np.save("y_test.npy", y_test)


In [ ]:
import numpy as np
import tifffile

# Data Preprocessing
def preprocess_data(X, y):
    # Normalization
    X = X / 255.0  # Assuming pixel values range from 0 to 255

    # Convert to torch tensors
    X_tensor = torch.from_numpy(X).float()
    y_tensor = torch.from_numpy(y).long()  # Assuming integer labels

    return X_tensor, y_tensor

# Load the split datasets
X_train = tifffile.imread("X_train.tif")
X_test = tifffile.imread("X_test.tif")
y_train = tifffile.imread("y_train.tif")
y_test = tifffile.imread("y_test.tif")

# Preprocess the datasets
X_train, y_train = preprocess_data(X_train, y_train)
X_test, y_test = preprocess_data(X_test, y_test)

# Print the shapes of the preprocessed datasets
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)


In [ ]:
print("Shape of X_train:", X_train.shape)

print("Shape of X_test:", X_test.shape)

In [ ]:

import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm

# Assuming you already have X_train, y_train, X_test, y_test
# Convert the data to PyTorch tensors and create DataLoader objects
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=22, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=22, shuffle=False)

# Initialize the model, loss function, and optimizer
# Initialize the model with the correct number of input channels
model = UNet(in_channels=X_train.size(2), out_channels=2)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Define the training function
def train(model, train_loader, criterion, optimizer, num_epochs=10):
    model.train()  # Set the model to train mode
    for epoch in range(num_epochs):
        running_loss = 0.0
        for inputs, targets in tqdm(train_loader, desc=f'Epoch {epoch + 1}/{num_epochs}', leave=False):
            optimizer.zero_grad()  # Zero the parameter gradients
            outputs = model(inputs)  # Forward pass
            loss = criterion(outputs, targets.long())  # Calculate the loss
            loss.backward()  # Backward pass
            optimizer.step()  # Optimize

            running_loss += loss.item() * inputs.size(0)

        epoch_loss = running_loss / len(train_loader.dataset)
        print(f'Train Loss: {epoch_loss:.4f}')

# Train the model
train(model, train_loader, criterion, optimizer, num_epochs=10)
